# TPC-H Query 5 &ndash; The Full Query

This IPython notebook is part of a series of tutorials that introduce how data algebra facilitates querying data from multiple sources and in different structures, on the example of a modified [TPC-H][] query. 

The tutorials assume basic familiarity with our library; we suggest working through our [Hello_World][] tutorial first. They also assume basic knowledge of relational data, RDF and XML. 

This tutorial assumes knowledge of concepts introduced in earlier parts, so it is best to work through them in the listed sequence:

- [1-Introduction][]: Introduces this series of tutorials, TPC-H, the TPC-H query 5 and our modifications to it.
- [2-Tables][]: Introduces our representation of tabular data, on the example of CSV.
- [3-Graphs][]: Introduces our representation of RDF-style tabular graph data, on the example of Turtle.
- [4-Hierarchies][]: Introduces our representation of hierarchical data, on the example of XML.
- **[5-Query][] (this tutorial)**: Brings it all together and explains the whole query.

[TPC-H]: <http://www.tpc.org/tpch/> (TPC-H Benchmark Main Page)
[Hello_World]: <../Hello_World.ipynb> (IPython Notebook: Hello World)
[1-Introduction]: <1-Introduction.ipynb> (IPython Notebook: TPC-H Query 5 - Introduction)
[2-Tables]: <2-Tables.ipynb> (IPython Notebook: TPC-H Query 5 - Tables)
[3-Graphs]: <3-Graphs.ipynb> (IPython Notebook: TPC-H Query 5 - Graphs)
[4-Hierarchies]: <4-Hierarchies.ipynb> (IPython Notebook: TPC-H Query 5 - Hierarchies)
[5-Query]: <5-Query.ipynb> (IPython Notebook: TPC-H Query 5 - Query)

To refresh your memory, here's the full version of the modified query 5 from [1-Introduction - Our Modifications][]:

``` sql
SELECT
    nationname, 
    SUM(lineitem.extendedprice * (1 - lineitem.discount)) AS revenue
FROM (
    SELECT 
        custkey, orderkey, suppkey, nationkey, nationname
    FROM (
        SELECT 
            custkey, suppkey, nationkey, nationname
        FROM (
            -- This is an SQL subquery. It extracts the customers in the selected 
            -- nations (which come from an XML pseudo-subquery) and returns the
            -- customer key 'custkey', nation key 'nationkey' and nation name 
            -- 'nationname'.
            SELECT 
                custkey, nationkey, nationname
            FROM 
                customer
            JOIN (
                -- This is a pseudo-subquery in XQuery. It extracts a list of nations
                -- for the desired region, and for every nation it returns an XML 'row' 
                -- with the columns 'nationkey' and 'nationname' (renamed from 'name').
                -- This forms an XML pseudo-'table' named 'nations' that is then joined 
                -- with 'customer'.
                for $x in doc("regions.xml")/regions/region[name="MIDDLE EAST"]/nation
                    return <nation>{$x/nationkey}<nationname>{data($x/name)}</nationname></nation>
            ) AS nations
            ON
                customer.nationkey = nations.nationkey
        ) AS customers_nations_projected
        JOIN (
            -- This is a pseudo-subquery in SPARQL. It extracts a table with the
            -- columns 'suppkey' and 'nationkey' from the RDF graph 'supplier'
            -- and injects them into the outer query as table 'supplier'.
            SELECT 
                ?suppkey, ?nationkey
            FROM 
                supplier
            WHERE {
                ?supplier <tpch:suppkey> ?suppkey .
                ?supplier <tpch:nationkey> ?nationkey .
            }
        ) AS supplier_solutions
        ON 
            supplier_solutions.nationkey = customers_joined_projected.nationkey
    ) AS result1
    JOIN (
        -- This is an SQL subquery. It filters the table 'orders' by the desired 
        -- date range and projects the columns 'orderkey' and 'custkey'.
        SELECT 
            orderkey, custkey
        FROM 
            orders
        WHERE 
            startdate <= orders.orderdate and orders.orderdate < enddate 
    ) AS orders_restricted_projected
    ON 
        result1.custkey = orders_restricted_projected.custkey
) AS result2
JOIN 
    lineitem
ON 
    lineitem.orderkey = result2.orderkey AND lineitem.suppkey = result2.suppkey
GROUP BY 
    nationname
```

The subqueries (and pseudo-subqueries) of it have been introduced in the previous notebooks of this series. In this notebook we bring it all together and run the full query.

[1-Introduction - Our Modifications]: <1-Introduction.ipynb#Our-Modifications> (IPython Notebook: TPC-H Query 5 - Introduction - Our Modifications)

# The Subqueries

## SQL Subqueries

The code from [2-Tables][] is collected in the following two functions that each returns a clan. Note how the `nations` table (for which we used an arbitrary clan in [2-Tables][]) has become an argument of the function `get_customers_nations_projected()`. Similarly, `startdate` and `enddate` (they are used in the restriction on the `orders` table) have become an argument of the function `get_orders_restricted_projected()`.

[2-Tables]: <2-Tables.ipynb> (IPython Notebook: TPC-H Query 5 - Tables)

In [1]:
from datetime import datetime
import algebraixlib.algebras.clans as clans
import algebraixlib.algebras.sets as sets
import algebraixlib.io.csv as csv

def get_customers_nations_projected(nations):
    customer_types = {'custkey': int, 'nationkey': int, 'acctbal': float}
    customers = csv.import_csv('customer.csv', customer_types)
    
    customers_nations = clans.functional_cross_union(customers, nations)
    customers_nations_projected = clans.project(customers_nations,
        'custkey', 'nationkey', 'nationname')
    return customers_nations_projected

def get_orders_restricted_projected(startdate, enddate):
    def read_date(date_str: str) -> datetime:
        return datetime.strptime(date_str, '%Y-%m-%d').date()
    orders_types = {
        'orderkey': int, 'custkey': int, 'orderdate': read_date,
        'totalprice': float, 'shippriority': int
    }
    orders = csv.import_csv('orders.csv', orders_types)
    
    def select_dates(rel) -> bool:
        orderdate = rel('orderdate').value
        return (startdate <= orderdate) and (orderdate < enddate)
    orders_restricted = sets.restrict(orders, select_dates)
    orders_restricted_projected = clans.project(orders_restricted, 'orderkey', 'custkey')
    return orders_restricted_projected

## SPARQL Subquery

The code from [3-Graphs][] is in the function `get_supplier_solutions()`; it represents the SPARQL pseudo-subquery that provides the supplier data.

[3-Graphs]: <3-Graphs.ipynb> (IPython Notebook: TPC-H Query 5 - Graphs)

In [2]:
import rdflib
import algebraixlib.io.rdf as rdf

def get_supplier_solutions():
    suppliers = rdf.import_graph('supplier.ttl')
    bgp_suppkey = clans.compose(
        clans.superstrict(suppliers, clans.from_dict({'p': rdflib.URIRef('tpch:suppkey')})), 
        clans.from_dict({'supplier': 's', 'suppkey': 'o'}))
    bgp_nationkey = clans.compose(
        clans.superstrict(suppliers, clans.from_dict({'p': rdflib.URIRef('tpch:nationkey')})),
        clans.from_dict({'supplier': 's', 'nationkey': 'o'}))
    supplier_solutions = clans.project(
        clans.functional_cross_union(bgp_suppkey, bgp_nationkey), 'nationkey', 'suppkey')
    return supplier_solutions

## XQuery Subquery

The code from [4-Hierarchies][] is in `get_nations()`. It returns a clan that associates `nationkey` and `nationname` for the nations in the region with the name `regionname`.

[4-Hierarchies]: <4-Hierarchies.ipynb> (IPython Notebook: TPC-H Query 5 - Hierarchies)

In [3]:
import algebraixlib.io.xml as xml

def get_nations(regionname):
    regions = xml.import_xml('regions.xml', convert_numerics=True)
    target_region = clans.superstrict(
        regions('regions')['region'], 
        clans.from_dict({'name': regionname}))
    nations = clans.compose(
        target_region['nation'], 
        clans.from_dict({'nationkey': 'nationkey', 'nationname': 'name'}))
    return nations

# Joining the Tables and Subquery Results

We work from the inside out here. The first join that doesn't have an embedded join is this one:

``` sql
(
    -- This is an SQL subquery. It extracts the customers in the selected 
    -- nations (which come from an XML pseudo-subquery) and returns the
    -- customer key 'custkey', nation key 'nationkey' and nation name 
    -- 'nationname'.
    SELECT 
        custkey, nationkey, nationname
    FROM 
        customer
    JOIN (
        -- This is a pseudo-subquery in XQuery. It extracts a list of nations
        -- for the desired region, and for every nation it returns an XML 'row' 
        -- with the columns 'nationkey' and 'nationname' (renamed from 'name').
        -- This forms an XML pseudo-'table' named 'nations' that is then joined 
        -- with 'customer'.
        for $x in doc("regions.xml")/regions/region[name="MIDDLE EAST"]/nation
            return <nation>{$x/nationkey}<nationname>{data($x/name)}</nationname></nation>
    ) AS nations
    ON
        customer.nationkey = nations.nationkey
) AS customers_nations_projected
```

This (inner) join has already been introduced in [2-Tables - The customer Table][] and is implemented by the function `get_customers_nations_projected()`. The only change required here is that the `nations` table, which was hardcoded in the example, is now provided by the XQuery statement that is implemented in `get_nations()`.

[2-Tables - The customer Table]: <2-Tables.ipynb#The-customer-Table> (IPython Notebook: TPC-H Query 5 - Tables)

In [4]:
from algebraixlib.util.latexprinter import iprint_latex

customers_nations_projected = get_customers_nations_projected(get_nations('MIDDLE EAST'))
iprint_latex('customers_nations_projected', short=True)

<IPython.core.display.Math object>

The next join is this one:

``` sql
(
    SELECT 
        custkey, suppkey, nationkey, nationname
    FROM (...) AS customers_nations_projected
    JOIN (...) AS supplier_solutions
    ON 
        supplier_solutions.nationkey = customers_joined_projected.nationkey
) AS result1
```

(The subqueries provided by already discussed functions are replaced by ellipses ('...').)

Such an inner join is implemented as functional cross-union by the function [`clans.functional_cross_union`][] ($\blacktriangledown$, see [2-Tables - The customer Table][]). With this, this join becomes:

[2-Tables - The customer Table]: <2-Tables.ipynb#The-customer-Table> (IPython Notebook: TPC-H Query 5 - Tables)

In [5]:
import algebraixlib.algebras.clans as clans

result1 = clans.functional_cross_union(
    get_supplier_solutions(), 
    customers_nations_projected)
iprint_latex('result1', short=True)

<IPython.core.display.Math object>

The next join adds order information:

``` sql
(
    SELECT 
        custkey, orderkey, suppkey, nationkey, nationname
    FROM (...) AS result1
    JOIN (...) AS orders_restricted_projected
    ON 
        result1.custkey = orders_restricted_projected.custkey
) AS result2
```

This looks very similar:

In [6]:
result2 = clans.functional_cross_union(
    result1,
    get_orders_restricted_projected(datetime(1994, 1, 1).date(), datetime(1997, 1, 1).date()))
iprint_latex('result2', short=True)

<IPython.core.display.Math object>

In the last of the joins, we add the lineitem information. This time, instead of joining with the result of a subquery, we join directly with a imported table. Importing a table is done in the same way we we showed twice in [2-Tables][]. (Because of the size of the data involved, this operation takes a few seconds.)

[2-Tables]: <2-Tables.ipynb> (IPython Notebook: TPC-H Query 5 - Tables)

In [7]:
lineitem_types = {
    'orderkey': int, 'suppkey': int, 'extendedprice': float, 'discount': float,
    'partkey': int, 'linenumber': int, 'quantity': int, 'tax': float,
}
result3 = clans.functional_cross_union(result2, csv.import_csv('lineitem.csv', lineitem_types))
iprint_latex('result3', short=True)

<IPython.core.display.Math object>

# Creating the Final Columns

Part of the final data is a `revenue` column that is calculated from `extendedprice` and `discount`. We create this column here with an operation that reflects a typical set builder notation:

$$
Result4 = \{rel \underset{f}{\cup} \{revenue{\mapsto}(rel(extendedprice) * (1 - rel(discount)))\}\ :\ rel \in Result3\}
$$

The operation $\underset{f}{\cup}$ is a union that is only defined if it maintains the functional character of the relation it operates on. This makes sure that we don't accidentally add a couplet with a left that already exists in the relation and change the character of the relation.

- `calc_revenue()` expects a relation as argument that contains couplets with the lefts `extendedprice` and `discount`. From these it calculates the revenue and returns a couplet with the left part `revenue` and the right part set to the calculated value.
- [`Set`][] accepts a generator as argument; this feature is used here to create a set builder. 
- [`relations.functional_add`][] implements the operation $\underset{f}{\cup}$ and adds a couplet to a relation if the couplet's left doesn't yet exist as left in the relation.

In the printed output, the `revenue` column is not visible, but you can see that the number of hidden columns (15 in the result above) has now increased to 16.

In [8]:
import algebraixlib.algebras.relations as relations
from algebraixlib.mathobjects import Couplet, Set

def calc_revenue(rel):
    return Couplet('revenue', rel('extendedprice').value * (1 - rel('discount').value))

result4 = Set(relations.functional_add(rel, calc_revenue(rel)) for rel in result3)
iprint_latex('result4', short=True)

<IPython.core.display.Math object>

Finally we remove the unnecessary columns, projecting only the desired columns `revenue` and `nationname`. (You see the typical rounding problems in the output that appear when working with floating-point types and exact values. We'll ignore this in this example; it is not relevant to the data algebra.)

In [9]:
revenue_by_nations = clans.project(result4, 'revenue', 'nationname')
iprint_latex('revenue_by_nations')

<IPython.core.display.Math object>

# Grouping and Aggregation

The final step of our query is the aggregation; we want to group the results by `nationname` and aggregate the `revenue` (as sum). 

As first step we partition `result5` by the column `nationname`. 

- [`algebraixlib.partition`][] contains our partition operations.
- [`partition.partition`][] partitions a clan. Its second argument is a function that returns same values for relations that should be considered part of the same equivalence class. Since we want to partition on the column `nationname`, we simply return its value.

In [10]:
import algebraixlib.partition as partition

revenue_grouped_by_nations = partition.partition(revenue_by_nations, lambda rel: rel('nationname'))
iprint_latex('revenue_grouped_by_nations')

<IPython.core.display.Math object>

Eventually, generic aggregation will be part of our library. It isn't yet, so all we have for now is this code here. 

- `aggregate()` aggregates on a 'horde' (a set of clans) that is the result of a partition operation on a clan and returns an 'aggregated' clan with the results of the aggregation. Its arguments are:

    - `group_left`: The left part of the couplets on which the partition is grouped (and on which the aggregation will also group). In our case that is `nationname`.
    - `aggregation_left`: The left part of the couplets that we want to aggregate. In our case that is `revenue`.
    - `aggregate_func`: A callable (Python function or similar) that has a member `.identity` (the identity element of the operation, 0 in our case) and accepts two arguments, on which it executes the aggregation operation (addition in our case).
    
- `aggregate_sum()` is the aggregation function that is used as `aggregate_func` argument to `aggregate()`.

In [11]:
def aggregate(horde, group_left, aggregation_left, aggregate_func):
    aggregation = {}
    for clan in horde:
        aggregation_value = aggregate_func.identity
        for relation in clan:
            aggregation_value = aggregate_func(aggregation_value,
                relation(aggregation_left).value)
        first_relation = next(iter(clan))
        aggregation[first_relation(group_left)] = aggregation_value
    return Set([Set(Couplet(group_left, key), Couplet(aggregation_left, aggregation[key]))
        for key in aggregation])

def aggregate_sum(arg1, arg2):
    return arg1 + arg2
aggregate_sum.identity = 0

Using the ad-hoc aggregation code from above, we can calculate the final result:

In [12]:
revenue_aggregate = aggregate(revenue_grouped_by_nations, 'nationname', 'revenue', aggregate_sum)
iprint_latex('revenue_aggregate')

<IPython.core.display.Math object>

# Final Considerations

With this example, we show how data from diverse sources and presented in different structures can be homogeneously represented and processed by data algebra. The unique advantage of doing so is that once the data is accessible to operations that follow the principles of data algebra, the form and sequence in which these operations are executed can be altered according to precisely defined mathematical rules, without changing the result. 

----
&copy; Copyright 2015 Algebraix Data Corporation

This file is part of [`algebraixlib`][] .

[`algebraixlib`][] is free software: you can redistribute it and/or modify it under the terms of [version 3 of the GNU Lesser General Public License][] as published by the [Free Software Foundation][].

[`algebraixlib`][] is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License along with [`algebraixlib`][]. If not, see [GNU licenses][].

[`algebraixlib`]: <http://github.com/AlgebraixData/algebraixlib> (A Python library for data algebra)
[Version 3 of the GNU Lesser General Public License]: <http://www.gnu.org/licenses/lgpl-3.0-standalone.html> 
[Free Software Foundation]: <http://www.fsf.org/>
[GNU licenses]: <http://www.gnu.org/licenses/>